# GTI770 - Systèmes Intelligents et Apprentissage Machine

### Alessandro L. Koerich

## Notebook Jupyter - 9_RNA_MLP_UpperLowercaseHandwriting_52Classes

##### Ver. 1: July 2018
##### Ver. 2: March 2019

In [ ]:
# Imports
import numpy as np
import time
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Imports from KERAS
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import TensorBoard

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.callbacks import TensorBoard

In [ ]:
# Imports from TENSOR FLOW
import tensorflow as tf

In [ ]:
# Testing the installation of Tensor Flow
hello = tf.constant('Hello, TensorFlow!')
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

In [ ]:
####### ATTENTION ######
# This part of the code is problematic and it may or may not work
# If you get stucked here during an execution, you must comment the last line of this cell # TB()
# The workaround is to open a terminal and run tensorboard in background....
#
# Open a Windows / Linux / MacOS terminal
#
# Activate the conda environment by issuing the following command:
# Windows terminal:
# C:> activate gti770
# (gti770)C:>  # Your prompt should change
# 
# Linux terminal / MacOS terminal:
# $ source activate gti770
# (gti770)$  # Your prompt should change
# Run Tensorboard in background by issuing the following command:
#(gti770)$ tensorboard --logdir="logs" &
# 
# Keep the terminal open and open a new tab in your brownser and type the address: http://127.0.0.1:6006


def TB(cleanup=False):
    import webbrowser
    webbrowser.open('http://127.0.0.1:6006')

    !tensorboard --logdir="logs"

    if cleanup:
        !rm -R logs/
# TB()

In [ ]:
# Code for avoiding keras + tensorflow from using all memory:
# Similar to the solution above, but also need to manually setup the session on Keras back-end:
import tensorflow as tf
# config = tf.ConfigProto(device_count = {'GPU': 2})
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)
####################

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# Load data from file
# NIST Train 52 Classes Uppercase + Lowecase Handwritten Characters
# 74,880 samples for training
# 23,670 samples for validation
# 23,941 samples for testing
# 108-dimensional feature vectors
# 26 classes (A-Z uppercase characters) + 26 classes (a-z lowercase characters) 

TrainData = np.loadtxt('CSV_Files/Char_UpperLower52.train.csv', delimiter=' ', dtype=np.str)
ValidData = np.loadtxt('CSV_Files/Char_UpperLower52.val.csv', delimiter=' ', dtype=np.str)
TestData  = np.loadtxt('CSV_Files/Char_UpperLower52.test.csv' , delimiter=' ', dtype=np.str)

Xtrain = TrainData[0:74779,0:108].astype(np.float)
Ytrain = TrainData[0:74779,108:160].astype(np.int)

Xvalid = ValidData[0:23669,0:108].astype(np.float)
Yvalid = ValidData[0:23669,108:160].astype(np.int)

Xtest  = TestData[0:23940,0:108].astype(np.float)
Ytest  = TestData[0:23940,108:160].astype(np.int)

In [ ]:
Xtrain
# 108 columns = inputs

In [ ]:
Ytrain
# 52 columns = outputs

In [ ]:
# normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
Xtrain = scaler.fit_transform(Xtrain)
Xvalid = scaler.fit_transform(Xvalid)
Xtest  = scaler.fit_transform(Xtest)

In [ ]:
num_classes = Ytrain.shape[1]
input_dim   = Xtrain.shape[1]

In [ ]:
input_dim

In [ ]:
num_classes

In [ ]:
def twolayer_model():
    print("Two-Layer NN\n")
    # create model
    model = Sequential()
    model.add(Dense(output_dim=num_classes, input_dim=input_dim))
    model.add(Activation('softmax'))
    model.summary()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
    return model

In [ ]:
def threelayer_model():
    # create model
    print("Three-Layer NN with 50 hidden neurons\n")
    model = Sequential()
    model.add(Dense(50, input_dim=input_dim))
    model.add(Activation('sigmoid'))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
    return model

In [ ]:
def fourlayer_model():
    # create model
    print("Four-Layer NN with 100 and 50 hidden neurons\n")
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim))
    model.add(Activation('sigmoid'))
    model.add(Dense(50))
    model.add(Activation('sigmoid'))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
    return model

In [ ]:
# build the model
# Choose only one model

# model = twolayer_model()
model = threelayer_model()
# model = fourlayer_model()

# Choose the number of learning cycles and the batch size
num_cycles = 1000
mini_batch = 500

In [ ]:
# Create variables to Tensorboard tracing
now = time.strftime("%c")
tbcallback = TensorBoard(log_dir='./logs/'+now, histogram_freq=0, write_graph=True, write_images=True )

In [ ]:
# Create a checkpoint to store the best model 

from keras.callbacks import ModelCheckpoint
from livelossplot    import PlotLossesKeras

# Checkpoints 
filepath   = "weights_RNA_MLP_ULH52.best.hdf5"
print( filepath )

checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max') 

callbacks_list = [tbcallback, checkpoint, PlotLossesKeras()]

In [ ]:
# Fit the model (TRAIN)
model.fit(Xtrain, Ytrain, validation_data=(Xvalid, Yvalid), epochs=num_cycles,
          batch_size=mini_batch, callbacks=callbacks_list )

In [ ]:
# Final evaluation of the model (On the Training, Validation or Test dataset)
scores = model.evaluate(Xtrain, Ytrain, verbose=0)
print("Error on the training dataset: %.2f%%" % (100-scores[1]*100))
scores = model.evaluate(Xvalid, Yvalid, verbose=0)
print("Error on the validation dataset: %.2f%%" % (100-scores[1]*100))
scores = model.evaluate(Xtest, Ytest, verbose=0)
print("Error on the test dataset: %.2f%%" % (100-scores[1]*100))

In [ ]:
def get_confusion_matrix_one_hot(model_results, truth):
    '''model_results and truth should be for one-hot format, i.e, have >= 2 columns,
    where truth is 0/1, and max along each row of model_results is model result
    '''
    
    assert model_results.shape == truth.shape
    num_outputs = truth.shape[1]
    confusion_matrix = np.zeros((num_outputs, num_outputs), dtype=np.int32)
    predictions = np.argmax(model_results,axis=1)
    assert len(predictions)==truth.shape[0]

    for actual_class in range(num_outputs):
        idx_examples_this_class = truth[:,actual_class]==1
        prediction_for_this_class = predictions[idx_examples_this_class]
        for predicted_class in range(num_outputs):
            count = np.sum(prediction_for_this_class==predicted_class)
            confusion_matrix[actual_class, predicted_class] = count
    assert np.sum(confusion_matrix)==len(truth)
    assert np.sum(confusion_matrix)==np.sum(truth)
    return confusion_matrix

In [ ]:
# Predict and show the confusion matrix (For the Validation dataset)
predict = model.predict(Xvalid)
confusion_matrix = get_confusion_matrix_one_hot(predict, Yvalid)

In [ ]:
df_cm = pd.DataFrame(confusion_matrix, index = [i for i in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"], columns = [i for i in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"])
plt.figure(figsize = (20,20))
sn.heatmap(df_cm, annot=True)
plt.show()

In [ ]:
print("Notebook ended")